# References

- [Chapters 8,9,12] https://nostarch.com/practical-deep-learning-python
- [MNIST Example Code] https://github.com/pytorch/examples/blob/main/mnist/main.py
- [CNN Tutorial] https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
- [Pytorch Tutorial] https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [3]:
# enable cuda
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 layer in, 32 layers out, 32 filters, 3x3 kernel with 1 stride
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 32 layers in, 64 layers out, 3x3 kernel with 1 stride
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # 25% dropout
        self.dropout1 = nn.Dropout(0.25)
        # 50% dropout
        self.dropout2 = nn.Dropout(0.5)
        # Fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        # Turn into classifications
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Layer 0: raw data in : 28x28x1 in
        # ---
        # Layer 1: convolution : 26x26x32 out
        x = self.conv1(x)
        # Nonlinearity function : --
        x = F.relu(x)
        # Layer 2: convolution : 24x24x64 out
        x = self.conv2(x)
        # Nonlinearity function : --
        x = F.relu(x)
        # Pooling 2x2, take max, stride = kernel = 2 : 12x12x64 out
        x = F.max_pool2d(x, 2)
        # 25% dropout
        x = self.dropout1(x)
        # Unwrap layers : 12 * 12 * 64 = 9,216 1d-vector out
        x = torch.flatten(x, 1)
        # Fully Connected NN : 9,216 * 128 = 1,179,648 weights + 128 biases : 128 out
        x = self.fc1(x)
        # Nonlinearity function : --
        x = F.relu(x)
        # 50% dropout
        x = self.dropout2(x)
        # NN : 128 * 10 = 1,280 weights + 10 biases : 10 out
        x = self.fc2(x)
        # Softmax 
        output = F.log_softmax(x, dim=1)
        return output

In [33]:
# Training/testing setup
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1_000}

cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': True}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

In [24]:
transform=transforms.Compose([
    transforms.ToTensor(), # convert to the tensor data structure
    transforms.Normalize((0.1307,), (0.3081,)) # these values are the mean and std, respectively
    ])

# load datasets
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

# calculate mean and standard dev of training set
mean = dataset1.train_data.float().mean()/255
std = dataset1.train_data.float().std()/255
print(f'{mean=},{std=}')

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


mean=tensor(0.1307),std=tensor(0.3081)


In [27]:
# create model on gpu
model = Net().to(device)

# Adadelta loss is what was used in the example code.
# Provides 99% accuracy, vs. 96% for Cross Entropy Loss
# https://pytorch.org/docs/stable/generated/torch.optim.Adadelta.html
# optimizer = optim.Adadelta(model.parameters(), lr=1.0)

# Cross Entropy Loss is what I read about first, so I wanted to test it out along with Stochastic gradient descent
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [35]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # Set all gradient params to 0
        optimizer.zero_grad()
        # Pass through training data
        output = model(data)
        # Calculate Cross Entropy Loss
        loss = criterion(output, target)

        # See note above about Adadelta
        # loss = F.nll_loss(output, target)

        # run backpropagation
        loss.backward()
        # update parameters
        optimizer.step()


def test(model, device, test_loader):
    # enable evaluation mode
    model.eval()
    test_loss = 0
    correct = 0
    # Disable gradient calculation - inference only mode
    with torch.no_grad():
        for data, target in test_loader:
            # move data to gpu
            data, target = data.to(device), target.to(device)
            # run inference
            output = model(data)
            # calculate loss by cross entropy loss
            test_loss += criterion(output, target)

            # See note above about Adadelta
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            
            # Find the most likely match
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            # Mark as correct if model guessed correctly
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [36]:
# scheduler decays learning rate parameters
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    # update the learning rate
    scheduler.step()
    print(f"Epoch {epoch} complete!")


Test set: Average loss: 0.0001, Accuracy: 9581/10000 (96%)

Epoch 0 complete!

Test set: Average loss: 0.0001, Accuracy: 9590/10000 (96%)

Epoch 1 complete!
